In [ ]:
#mecabで分かち書きしながら、tokenizeする


In [ ]:
import json
target_path="/data/hatakeyama/python/llm_corpus/corpus_scale_500.jsonl"
lines=[]
with open(target_path, "r") as f:
    for line in f:
        lines.append(json.loads(line)["text"])

with open("../data/corpus_text.txt","w") as f:
    f.write("")
with open("../data/corpus_text.txt","a") as f:
    for line in lines:
        f.write(line+"\n")

In [ ]:
#わかちがき
!mecab -F"%M||||" -E"\n" -b 100000 < ../data/corpus_text.txt  > ../data/corpus_text.txt.tok

In [ ]:
len(lines)

In [ ]:
from types import SimpleNamespace
import sentencepiece as spm
import sys
import yaml
import os
from special_token_list import *


def yaml_to_namespace(yaml_path):
    with open(yaml_path, 'r') as file:
        # YAMLファイルを辞書として読み込む
        data = yaml.safe_load(file)
        # 辞書をSimpleNamespaceに変換
        return recursive_namespace(data)


def recursive_namespace(data):
    if isinstance(data, dict):
        # 再帰的に辞書の各要素をSimpleNamespaceに変換
        return SimpleNamespace(**{k: recursive_namespace(v) for k, v in data.items()})
    elif isinstance(data, list):
        # リストの要素も変換
        return [recursive_namespace(v) for v in data]
    else:
        # その他のデータ型はそのまま返す
        return data


args = yaml_to_namespace('../30tokenize/config.yaml')

args.input="../data/corpus_text.txt.tok"
#args.vocab_size=3000

In [ ]:
spm.SentencePieceTrainer.train(
        input=args.input,
        model_prefix=args.model_prefix,
        vocab_size=args.vocab_size,
        character_coverage=args.character_coverage,
        model_type=args.model_type,
        num_threads=args.num_threads,
        train_extremely_large_corpus=args.train_extremely_large_corpus,
        user_defined_symbols=[
            BOS_TOKEN,
            EOS_TOKEN,
            PAD_TOKEN,
            CLS_TOKEN,
            SEP_TOKEN,
            EOD_TOKEN,
            MASK_TOKEN,
            NEWLINE_TOKEN,
            EXTRA_TOKEN1,
            EXTRA_TOKEN2,
            EXTRA_TOKEN3,
            EXTRA_TOKEN4,
        ],  # Note: `NEWLINE_TOKEN` is needed in `user_defined_symbols`.
        byte_fallback=True,
        split_digits=True,
        allow_whitespace_only_pieces=True,
        remove_extra_whitespaces=False,
        pretokenization_delimiter="||||",
    )



In [ ]:
1